# Visualisierung der NR Wahlen

In [ ]:
import pandas as pd, datetime as dt, numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 128)

In [ ]:
results_mit_wk = pd.read_parquet("wahlergebnisse.parquet")
jahre = results_mit_wk.YEAR.unique()

### Bundesergebnisse

In [ ]:
x_ticks=jahre
y_ticks=list(range(0, 55, 5))
gemeindeergebnis = results_mit_wk["YEAR Partei StimmenMitWk StimmenGueltigMitWk".split()] \
    .groupby(["YEAR", "Partei"]).aggregate({"StimmenMitWk": "sum", "StimmenGueltigMitWk": "sum"})
gemeindeergebnis["Prozent"] = round(100 * gemeindeergebnis.StimmenMitWk / gemeindeergebnis.StimmenGueltigMitWk, 2)
gemeindeergebnis = gemeindeergebnis.Prozent.unstack()["SPÖ ÖVP FPÖ GRÜNE NEOS BZÖ".split()]
#gemeindeergebnis["FPÖVP"] = gemeindeergebnis["ÖVP"] + gemeindeergebnis["FPÖ"]
axis = gemeindeergebnis.plot.line(color={"SPÖ": "#ff0000", "ÖVP": "#000000", "FPÖ": "#0000ff", "GRÜNE": "#2dd22d",
                                  "NEOS": "#d22dca", "BZÖ": "#ff8000",
                                  "Linksp": "#ff6666", "FPÖVP": "#001166"},
                           title="NR Wahlen in Österreich seit 1995", xticks=x_ticks, yticks=y_ticks, grid=True, figsize=(10,5))
axis.legend(loc='upper center', ncols=6)
axis.figure.savefig(f"../nr_wahlen_{dt.datetime.now():%H%M}.svg")

### Gemeindeergebnisse

In [ ]:
x_ticks=jahre
y_ticks=list(range(0, 55, 5))
gemeindeergebnis = results_mit_wk[results_mit_wk.GKZ_BEV == 30639]["YEAR Partei StimmenMitWk StimmenGueltigMitWk".split()].set_index(["YEAR", "Partei"])
gemeindeergebnis["Prozent"] = round(100 * gemeindeergebnis.StimmenMitWk / gemeindeergebnis.StimmenGueltigMitWk, 2)
gemeindeergebnis = gemeindeergebnis.Prozent.unstack()["SPÖ ÖVP FPÖ GRÜNE NEOS".split()]
#gemeindeergebnis["FPÖVP"] = gemeindeergebnis["ÖVP"] + gemeindeergebnis["FPÖ"]
gemeindeergebnis.plot.line(color={"SPÖ": "#ff0000", "ÖVP": "#000000", "FPÖ": "#0000ff", "GRÜNE": "#2dd22d",
                                  "NEOS": "#d22dca", "BZÖ": "#ff8000",
                                  "Linksp": "#ff6666", "FPÖVP": "#001166"},
               title="NR Wahlen in Traiskirchen seit 1995", xticks=x_ticks, yticks=y_ticks, grid=True, figsize=(10,5));

## Wahlberechtigte in Bezirken

In [ ]:
import geopandas as gp  # https://geopandas.org/en/stable/docs/user_guide.html

In [ ]:
bev_shapefile = "VGD.shp"    # Verwaltungsgrenzen aus dem BEV Katalog
bezirke_shapefile = "STATISTIK_AUSTRIA_POLBEZ_20230101.shp"    # Bezirksgrenzen von Statistik Austria (für die Wiener Bezirke)
# Aus dem prj File der Verwaltungsgrenzen.
crs = 'PROJCS["MGI_Austria_Lambert",GEOGCS["GCS_MGI",DATUM["D_MGI",SPHEROID["Bessel_1841",6377397.155,299.1528128]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",400000.0],PARAMETER["False_Northing",400000.0],PARAMETER["Central_Meridian",13.33333333333333],PARAMETER["Standard_Parallel_1",46.0],PARAMETER["Standard_Parallel_2",49.0],PARAMETER["Latitude_Of_Origin",47.5],UNIT["Meter",1.0]]'

In [ ]:
# Gemeinden ohne Wien lesen. Wien hat Gemeindebezirke, diese sind aber hier nicht enthalten.
katastralgemeinden = gp.read_file(bev_shapefile, encoding="utf-8") \
    [['KG_NR', 'GKZ', 'BKZ', 'BL_KZ', 'KG', 'PG', 'PB', 'BL', 'geometry']] \
    .rename({'BL_KZ': 'BLKZ'}, axis=1) \
    .astype({"KG_NR": int, "GKZ": int, "BKZ": int, "BLKZ": int, "KG": "string", "PG": "string", "PB": "string", "BL": "string"}) \
    .query("GKZ < 90000") \
    .set_crs(crs, allow_override=True)
katastralgemeinden["PB"] = katastralgemeinden.PB.str.replace("(", " (", regex=False)
# Gemeindebezirke von Wien lesen. Wir bilden diese Bezirke als eigene Gemeinde ab.
bezirke_wien = gp.read_file(bezirke_shapefile, encoding="utf-8", crs="EPSG:31287") \
    .rename({'g_id': "BKZ", "g_name": "PG"}, axis=1) \
    .astype({"BKZ": int, "PG": "string"}) \
    .query("BKZ >= 901") \
    .set_crs(crs, allow_override=True)
bezirke_wien["GKZ"] = bezirke_wien.BKZ * 100 + 1
bezirke_wien["PG"] = bezirke_wien.PG.str.replace(",", ", ", regex=False)
bezirke_wien["PG"] = bezirke_wien.PG.str.replace("\s+", " ", regex=True)
bezirke_wien["PB"] = bezirke_wien.PG
bezirke_wien["BLKZ"] = 9
bezirke_wien["BL"] = "Wien"
bezirke_wien["BL"] = bezirke_wien.BL.astype("string")
gemeinden_shp = katastralgemeinden.dissolve(by="GKZ").drop(["KG_NR", "KG"], axis=1).reset_index()
gemeinden_shp = pd.concat([gemeinden_shp, bezirke_wien])
bezirke = gemeinden_shp.dissolve(by="BKZ").drop(["GKZ", "PG"], axis=1).reset_index().set_index("BKZ")


In [ ]:
wahlberechtigte = results_mit_wk.groupby(["YEAR", "GKZ_BEV"], as_index=False).first() \
    .query("(YEAR == 1995) | (YEAR == 2019)") \
    .groupby(["BKZ_BEV", "YEAR"]).aggregate(Wahlberechtigte=("Wahlberechtigte", "sum")).Wahlberechtigte.unstack() \
    .join(bezirke).astype({1995: int, 2019: int}).rename({1995: "YEAR1995", 2019: "YEAR2019"}, axis=1)
wahlberechtigte["DIFF"] = round(100*(wahlberechtigte["YEAR2019"] - wahlberechtigte["YEAR1995"])/wahlberechtigte["YEAR1995"], 2)
wahlberechtigte_shp = gp.GeoDataFrame(wahlberechtigte.reset_index(), crs=crs)
wahlberechtigte_shp.dtypes
wahlberechtigte_shp.to_file("wahlberechtigte.shp")